# Обучение модели CRNN для распознавания капчи

Этот notebook предназначен для обучения модели на Google Colab.

## Шаги:
1. Установка зависимостей
2. Загрузка файлов проекта
3. Генерация/загрузка датасета
4. Обучение модели
5. Сохранение результатов


In [1]:
# Установка зависимостей
%pip install captcha>=0.7.1 tqdm Pillow torch torchvision matplotlib google


## Загрузка файлов проекта

Вы можете загрузить файлы проекта несколькими способами:

### Вариант 1: Загрузить с GitHub (если проект в репозитории)
### Вариант 2: Загрузить файлы вручную через файловую систему Colab
### Вариант 3: Создать файлы прямо в Colab (клетки ниже)


In [2]:
# Загружаем файлы model.py, dataset_loader.py, train.py
# Если файлы загружены через Drive или GitHub, раскомментируйте соответствующие строки

# Для загрузки с GitHub:
!git clone https://github.com/alexeii89/captcha_ii.git
%cd captcha_ii

# Или загрузите файлы через интерфейс Colab (Files -> Upload)
# Затем скопируйте в рабочую директорию:
!cp model.py dataset_loader.py train.py main.py ./

print("Файлы должны быть в текущей директории")
!ls -la *.py 2>/dev/null || echo "Файлы .py не найдены. Загрузите их через интерфейс Colab или используйте следующий вариант."


Cloning into 'captcha_ii'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 20 (delta 7), reused 18 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 75.94 KiB | 25.31 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/captcha_ii
cp: 'model.py' and './model.py' are the same file
cp: 'dataset_loader.py' and './dataset_loader.py' are the same file
cp: 'train.py' and './train.py' are the same file
cp: 'main.py' and './main.py' are the same file
Файлы должны быть в текущей директории
-rw-r--r-- 1 root root  7548 Dec 11 06:39 dataset_loader.py
-rw-r--r-- 1 root root  3259 Dec 11 06:39 main.py
-rw-r--r-- 1 root root  5126 Dec 11 06:39 model.py
-rw-r--r-- 1 root root  3521 Dec 11 06:39 predict.py
-rw-r--r-- 1 root root 10166 Dec 11 06:39 train.py


## Вариант: Создать файлы прямо здесь

Если файлы не загружены, можно прочитать их из локальных файлов и создать в Colab.


## Генерация датасета

**Важно:** Если вы только что создали файл `main.py` в ячейке выше, убедитесь, что она выполнена успешно!

Теперь сгенерируем датасет. Вы можете:
1. Сгенерировать на Colab (займет время) - ячейка 10
2. Загрузить готовый датасет с Google Drive - ячейка 11 (опционально)


In [3]:
HOST = input("Введите IP или домен сервера: ")
USER = input("Введите имя пользователя: ")
REMOTE_PATH = input("Введите путь на сервере: ")
PASSWORD = input("Введите пароль: ")

In [5]:
# Вариант 1: Генерация датасета прямо в Colab

# Проверяем, что файл main.py существует и содержит правильную функцию
import os
if not os.path.exists('main.py'):
    raise FileNotFoundError("Файл main.py не найден! Сначала запустите ячейку 8, где создается main.py")

# Импортируем и перезагружаем модуль (на случай, если он уже был загружен)
import importlib
import sys

# Удаляем модуль из кэша, если он был загружен ранее
if 'main' in sys.modules:
    del sys.modules['main']

# Импортируем модуль заново
from main import generate_captcha_dataset

# Генерируем датасет
# Для быстрого теста можно использовать меньше изображений (например, 1000)
# Для полного обучения рекомендуется 50000
print("Начинаем генерацию датасета...")
generate_captcha_dataset()
print("\n✓ Генерация завершена!")


Начинаем генерацию датасета...
Начинаю генерацию 50000 изображений капчи...
Символы: АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ0123456789
Длина текста: 4-6 символов
Папка для сохранения: dataset


Генерация капчи: 100%|██████████| 50000/50000 [12:13<00:00, 68.13it/s]


✓ Готово! Сгенерировано 50000 изображений в папке 'dataset'
✓ Уникальных файлов: 50000

✓ Генерация завершена!


In [6]:
# Импортируем необходимые модули
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from model import CRNN, decode_predictions
from dataset_loader import get_data_loaders
import matplotlib.pyplot as plt
import os


In [7]:
# Функции для обучения (можно вынести в train.py, но для простоты оставим здесь)

def train_epoch(model, train_loader, criterion, optimizer, device, dataset):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    progress_bar = tqdm(train_loader, desc="Обучение")
    for images, targets, target_lengths, texts in progress_bar:
        images = images.to(device)
        targets = targets.to(device)
        target_lengths = target_lengths.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        outputs = outputs.permute(1, 0, 2)
        
        input_lengths = torch.full(
            size=(outputs.size(1),),
            fill_value=outputs.size(0),
            dtype=torch.long
        ).to(device)
        
        loss = criterion(outputs, targets, input_lengths, target_lengths)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        with torch.no_grad():
            pred_outputs = model(images)
            decoded_preds = decode_predictions(pred_outputs, dataset)
            
            for pred, true in zip(decoded_preds, texts):
                if pred == true:
                    correct += 1
                total += 1
        
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100 * correct / total:.2f}%'
        })
    
    return total_loss / len(train_loader), 100 * correct / total if total > 0 else 0


def validate(model, val_loader, criterion, device, dataset):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc="Валидация")
        for images, targets, target_lengths, texts in progress_bar:
            images = images.to(device)
            targets = targets.to(device)
            target_lengths = target_lengths.to(device)
            
            outputs = model(images)
            outputs = outputs.permute(1, 0, 2)
            
            input_lengths = torch.full(
                size=(outputs.size(1),),
                fill_value=outputs.size(0),
                dtype=torch.long
            ).to(device)
            
            loss = criterion(outputs, targets, input_lengths, target_lengths)
            total_loss += loss.item()
            
            pred_outputs = model(images)
            decoded_preds = decode_predictions(pred_outputs, dataset)
            
            for pred, true in zip(decoded_preds, texts):
                if pred == true:
                    correct += 1
                total += 1
            
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100 * correct / total:.2f}%'
            })
    
    return total_loss / len(val_loader), 100 * correct / total if total > 0 else 0


In [8]:
# Настройки обучения
# Исключаем Ь, Ъ, Й, О, Щ, З - их нет в капче
russian_letters = 'АБВГДЕЖИКЛМНПРСТУФХЦЧШЫЭЮЯ'
numbers = '1234567890'
characters = russian_letters + numbers

EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.001
DATASET_DIR = 'dataset'

print("=" * 60)
print("Настройки обучения")
print("=" * 60)
print(f"Эпох: {EPOCHS}")
print(f"Размер батча: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Количество символов: {len(characters)}")
print("=" * 60)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

# Загружаем данные
print("\nЗагрузка датасета...")
train_loader, val_loader, dataset = get_data_loaders(
    dataset_dir=DATASET_DIR,
    characters=characters,
    batch_size=BATCH_SIZE,
    train_split=0.8,
    num_workers=2  # Для Colab
)

print(f"Обучающих примеров: {len(train_loader.dataset)}")
print(f"Валидационных примеров: {len(val_loader.dataset)}")

# Создаем модель
model = CRNN(num_classes=len(characters) + 1).to(device)
print(f"\nМодель создана. Параметров: {sum(p.numel() for p in model.parameters()):,}")

# Функция потерь и оптимизатор
criterion = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

# История
train_losses, train_accs = [], []
val_losses, val_accs = [], []

os.makedirs('models', exist_ok=True)
best_val_acc = 0

print("\nНачинаем обучение...\n")


Настройки обучения
Эпох: 50
Размер батча: 32
Learning rate: 0.001
Количество символов: 34
Используемое устройство: cpu

Загрузка датасета...
Загружено 50000 изображений
Количество символов: 34
Обучающих примеров: 40000
Валидационных примеров: 10000

Модель создана. Параметров: 8,726,307

Начинаем обучение...



In [10]:
# Цикл обучения
for epoch in range(EPOCHS):
    print(f"\nЭпоха {epoch + 1}/{EPOCHS}")
    print("-" * 60)
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, dataset)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    val_loss, val_acc = validate(model, val_loader, criterion, device, dataset)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    old_lr = optimizer.param_groups[0]['lr']
    scheduler.step(val_loss)
    new_lr = optimizer.param_groups[0]['lr']
    
    print(f"\nРезультаты эпохи {epoch + 1}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    if old_lr != new_lr:
        print(f"  Learning rate: {old_lr:.6f} -> {new_lr:.6f}")
    
    # Сохраняем лучшую модель
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'characters': characters,
        }, 'models/best_model.pth')
        print(f"  ✓ Сохранена лучшая модель (Val Acc: {val_acc:.2f}%)")
    
    # Чекпоинт каждые 10 эпох
    if (epoch + 1) % 10 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_losses': train_losses,
            'val_losses': val_losses,
            'train_accs': train_accs,
            'val_accs': val_accs,
            'characters': characters,
        }, f'models/checkpoint_epoch_{epoch + 1}.pth')

# Финальное сохранение
torch.save({
    'epoch': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses,
    'train_accs': train_accs,
    'val_accs': val_accs,
    'characters': characters,
}, 'models/final_model.pth')

print("\n" + "=" * 60)
print("Обучение завершено!")
print(f"Лучшая точность на валидации: {best_val_acc:.2f}%")
print("=" * 60)



Эпоха 1/50
------------------------------------------------------------


Обучение:   0%|          | 0/1250 [00:00<?, ?it/s]


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/content/captcha_ii/dataset_loader.py", line 81, in __getitem__
    return self.subset[idx]
           ~~~~~~~~~~~^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataset.py", line 408, in __getitem__
    return self.dataset[self.indices[idx]]
           ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
  File "/content/captcha_ii/dataset_loader.py", line 50, in __getitem__
    target = [self.char_to_idx[char] for char in text]
              ~~~~~~~~~~~~~~~~^^^^^^
KeyError: '

In [ ]:
# Строим графики
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(train_losses, label='Train Loss')
ax1.plot(val_losses, label='Val Loss')
ax1.set_xlabel('Эпоха')
ax1.set_ylabel('Loss')
ax1.set_title('История Loss')
ax1.legend()
ax1.grid(True)

ax2.plot(train_accs, label='Train Acc')
ax2.plot(val_accs, label='Val Acc')
ax2.set_xlabel('Эпоха')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('История Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig('models/training_history.png')
plt.show()

print("График сохранен: models/training_history.png")


## Архивация и загрузка по SSH

Архивируйте папку models и загрузите на удаленный сервер по SSH.


In [ ]:
# Архивация и загрузка models по SSH

import os
import zipfile
import subprocess
from datetime import datetime

# ⚙️ ИЗМЕНИТЕ ЭТИ НАСТРОЙКИ НА СВОИ!
       # Куда сохранить на сервере
PORT = 22                          # SSH порт                 # Пароль (если нужен) или None для SSH ключей

# ============================================================================

print("=" * 70)
print("📦 АРХИВАЦИЯ И ЗАГРУЗКА MODELS ПО SSH")
print("=" * 70)

# 1. Создаем архив
print("\n1️⃣ Создание архива...")
models_dir = './models'

if not os.path.exists(models_dir):
    print("❌ Папка models не найдена!")
    print("Сначала запустите обучение модели.")
else:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archive_name = f'models_{timestamp}.zip'
    
    print(f"📦 Создание архива: {archive_name}")
    with zipfile.ZipFile(archive_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(models_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, '.')
                zipf.write(file_path, arcname)
                print(f"  ✅ {arcname}")
    
    file_size = os.path.getsize(archive_name) / (1024 * 1024)
    print(f"\n✅ Архив создан: {archive_name} ({file_size:.2f} MB)")
    
    # 2. Загружаем на сервер
    if HOST == 'your-server.com':
        print("\n⚠️  ВНИМАНИЕ: Настройте параметры SSH!")
        print("Измените значения в начале скрипта:")
        print("  - HOST: адрес вашего сервера")
        print("  - USER: имя пользователя")
        print("  - REMOTE_PATH: путь на сервере")
        print("  - PASSWORD: пароль (если используете)")
    else:
        print(f"\n2️⃣ Загрузка на {USER}@{HOST}...")
        print(f"   Путь на сервере: {REMOTE_PATH}")
        
        if PASSWORD:
            # Устанавливаем sshpass если нужно
            try:
                subprocess.run(['which', 'sshpass'], check=True,
                             stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            except:
                print("📥 Установка sshpass...")
                subprocess.run(['apt-get', 'install', '-y', 'sshpass'], check=True)
            
            cmd = ['sshpass', '-p', PASSWORD, 'scp', '-P', str(PORT),
                   '-o', 'StrictHostKeyChecking=no', archive_name,
                   f'{USER}@{HOST}:{REMOTE_PATH}']
        else:
            # Используем SSH ключи
            cmd = ['scp', '-P', str(PORT), '-o', 'StrictHostKeyChecking=no',
                   archive_name, f'{USER}@{HOST}:{REMOTE_PATH}']
        
        try:
            subprocess.run(cmd, check=True)
            print(f"✅ Успешно загружено!")
            print(f"\n📁 Расположение на сервере: {REMOTE_PATH}{archive_name}")
            print(f"\n💡 Для распаковки на сервере выполните:")
            print(f"   ssh {USER}@{HOST} 'cd {REMOTE_PATH} && unzip -q {archive_name}'")
        except subprocess.CalledProcessError as e:
            print(f"❌ Ошибка загрузки!")
            print(f"\n💡 Проверьте:")
            print("   - Правильность HOST, USER, REMOTE_PATH")
            print("   - Наличие SSH ключей или правильность пароля")
            print("   - Доступность сервера и порта")
            print("   - Права доступа к REMOTE_PATH на сервере")
            print(f"\nОшибка: {e}")
        except Exception as e:
            print(f"❌ Ошибка: {e}")

print("\n" + "=" * 70)
